In [4]:
%matplotlib inline

import serial
import serial.tools.list_ports
import time
import numpy as np
import pyvista as pv
from pyvista import examples
import pyvistaqt as pvqt
from scipy.spatial import KDTree
import noise
from binary_to_csv import binary_to_csv
import os
import pandas as pd




In [34]:
DATA_DIR = '../data'
BIN_FILE_NAME = "data_20241015_183835.bin"

BIN_FILE_PATH = os.path.join(DATA_DIR, BIN_FILE_NAME)
CSV_FILE_PATH = os.path.join(DATA_DIR, BIN_FILE_NAME.replace(".bin", ".csv"))


binary_to_csv(BIN_FILE_PATH, CSV_FILE_PATH)
print(CSV_FILE_PATH)

df = pd.read_csv(CSV_FILE_PATH, header=None)
df

../data/data_20241015_183835.csv


,0,1,2,3,4,5,6,7,8,9,...,242,243,244,245,246,247,248,249,250,251
0,12026,12146,12195,7700,11000,6650,3713,8060,11381,8158,...,5595,4949,5279,5441,1395,5540,4835,6138,4396,2115
1,12033,12162,12190,7709,10998,6661,3707,8060,11378,8164,...,5582,4956,5263,5443,1390,5546,4832,6141,4390,2116
2,12028,12158,12203,7699,10999,6658,3723,8054,11376,8165,...,5579,4947,5271,5447,1374,5546,4830,6139,4382,2115
3,12026,12162,12197,7709,10992,6655,3712,8061,11374,8161,...,5586,4956,5279,5450,1385,5545,4831,6136,4385,2114
4,12034,12171,12202,7693,11004,6667,3723,8044,11383,8172,...,5577,4931,5264,5448,1381,5537,4827,6135,4388,2111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,12016,12157,12206,7702,10987,6660,3728,8059,11372,8158,...,5587,4905,5239,5446,1373,5530,4840,6130,4433,2105
112,12012,12164,12198,7707,10976,6660,3714,8063,11366,8161,...,5587,4917,5238,5450,1368,5534,4836,6139,4431,2113
113,12018,12166,12193,7707,10986,6669,3712,8059,11369,8165,...,5590,4913,5242,5446,1373,5537,4833,6131,4435,2117
114,12023,12162,12194,7699,10996,6661,3707,8061,11381,8163,...,5572,4913,5238,5447,1369,5535,4832,6129,4430,2115


In [53]:

ASSETS_DIR = '../assets'


N_COLUMNS = 21
N_ROWS = 12


points = np.load(os.path.join(ASSETS_DIR, "boob_grid_12_21.npy"))

# Remove points that are outside of the boob mesh.
# Because those points have been skipped during the points mapping, their value is (0, 0, 0)
origin = np.array([0, 0, 0])
points_to_skip = np.where((points == origin).all(axis=1))[0]
points_of_interest = np.delete(points, points_to_skip, axis=0)

# Create the point cloud and initialize the scalars
point_cloud = pv.PolyData(points_of_interest)
point_cloud['pressure'] = np.zeros(points_of_interest.shape[0])

pl = pv.Plotter( border=False)

pl.subplot(0, 0)
pl.add_title("Points pressed", font_size=12)
pl.add_mesh(point_cloud, scalars='pressure', cmap='cool', point_size=15)

pl.link_views()

pl.camera_position = 'xy'
pl.camera.elevation += 100

df_poi = df.drop(columns=points_to_skip)

thresholds = df_poi.iloc[0] + 100
points_touched = (df_poi > thresholds).any().values

point_cloud['pressure'][:] = points_touched
# point_cloud['pressure'][:] = df_poi.sum(axis=0).values

# pl.update_scalar_bar_range([np.min(df_poi.sum(axis=0).values), np.max(df_poi.sum(axis=0).values)])


pl.remove_scalar_bar()
pl.show()

print([np.min(df_poi.sum(axis=0).values), np.max(df_poi.sum(axis=0).values)])



Widget(value='<iframe src="http://localhost:50086/index.html?ui=P_0x3284c8c50_42&reconnect=auto" class="pyvist…

[89029, 7364529]


In [ ]:
# pl.show()

# def callback(step):
#     # pressure_values = df[step].values
#     print(step)
#     # Update the pressure values in the point cloud
#     # point_cloud['pressure'] = pressure_values[:len(point_cloud.points)]
#     # point_cloud['max_pressure'] = np.maximum(point_cloud['max_pressure'], point_cloud['pressure'])

#     # # Update the pressure values in the boob mesh using the normalized weights
#     # interpolated_pressure = np.sum(normalized_weights * point_cloud['pressure'][boob_mesh['nearest_points']], axis=1)
#     # boob_mesh['pressure'] = interpolated_pressure
#     # boob_mesh['max_pressure'] = np.maximum(boob_mesh['max_pressure'], boob_mesh['pressure'])


# pl.add_timer_event(max_steps=200, duration=10, callback=callback)
# pl.show()
# for index, row in df.iterrows():
#     # Extract the pressure values from the dataframe row
#     pressure_values = row.values

#     # Update the pressure values in the point cloud
#     point_cloud['pressure'] = pressure_values[:len(point_cloud.points)]
#     point_cloud['max_pressure'] = np.maximum(point_cloud['max_pressure'], point_cloud['pressure'])

#     # Update the pressure values in the boob mesh using the normalized weights
#     interpolated_pressure = np.sum(normalized_weights * point_cloud['pressure'][boob_mesh['nearest_points']], axis=1)
#     boob_mesh['pressure'] = interpolated_pressure
#     boob_mesh['max_pressure'] = np.maximum(boob_mesh['max_pressure'], boob_mesh['pressure'])

#     # Update the plot
#     # pl.update_scalar_bar_range([MIN_VALUE, MAX_VALUE])
#     time.sleep(0.1)  # Add a small delay to visualize the updates